# Caso: Analizando el nivel de CO2 en el aire

### Cargando los datos

In [ ]:
import pandas as pd
co2_levels = pd.read_csv('co2_levels.csv')
print(co2_levels)

In [ ]:
print(co2_levels.dtypes)

### Es necesario transformar el DataFrame en una serie de tiempo

In [ ]:
co2_levels['datestamp'] = pd.to_datetime(co2_levels.datestamp)

In [ ]:
co2_levels.set_index('datestamp',inplace=True)

### Revisamos si existen valores missing

In [ ]:
print(co2_levels.isnull().sum())

### Imputamos utilizando el siguiente dato válido (bfill)

In [ ]:
co2_levels = co2_levels.fillna(method='bfill')

In [ ]:
print(co2_levels.isnull().sum())

### Descomponemos la Serie con seasonal_decompose

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 11, 24
decomposition = sm.tsa.seasonal_decompose(co2_levels['co2'])
fig = decomposition.plot()
plt.show()

### Construimos un intervalo utilizando medias y desviaciones móviles

In [ ]:
rcParams['figure.figsize'] = 11, 8

# Compute the 52 weeks rolling mean of the co2_levels DataFrame
ma = co2_levels.rolling(window=52).mean()

# Compute the 52 weeks rolling standard deviation of the co2_levels DataFrame
mstd = co2_levels.rolling(window=52).std()

# Add the upper bound column to the ma DataFrame
ma['upper'] = ma['co2'] + (mstd['co2'] * 2)

# Add the lower bound column to the ma DataFrame
ma['lower'] = ma['co2'] - (mstd['co2'] * 2)

# Plot the content of the ma DataFrame
ax = ma.plot(linewidth=0.8, fontsize=6)

# Specify labels, legend, and show the plot
ax.set_xlabel('Date', fontsize=10)
ax.set_ylabel('CO2 levels in Mauai Hawaii', fontsize=10)
ax.set_title('Rolling mean and variance of CO2 levels\nin Mauai Hawaii from 1958 to 2001', fontsize=10)
plt.show();

### Ahora probamos generando varios modelos de tipo SARIMA para hacer los pronósticos

In [ ]:
import itertools

# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
import warnings
import numpy as np
warnings.filterwarnings("ignore") # specify to ignore warning messages

best_aic = np.inf
best_pdq = None
best_seasonal_pdq = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        
        try:
            model = sm.tsa.statespace.SARIMAX(co2_levels,
                                             order = param,
                                             seasonal_order = param_seasonal,
                                             enforce_stationarity=False,
                                             enforce_invertibility=False)
            results = model.fit()

            # print("SARIMAX{}x{}12 - AIC:{}".format(param, param_seasonal, results.aic))
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
        except:
            continue
print("Best SARIMAX{}x{}12 model - AIC:{}".format(best_pdq, best_seasonal_pdq, best_aic))

### Elegimos el mejor modelo de acuerdo a la métrica AIC

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(co2_levels,
                                      order=(1, 1, 1),
                                      seasonal_order=(1, 0, 1, 12),
                                      enforce_stationarity=False,
                                      enforce_invertibility=False)
results = best_model.fit()

In [ ]:
print(results.summary().tables[0])
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(15,12))
plt.show()

### Generamos el pronóstico para los siguientes meses

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2002-01-05'), end=pd.to_datetime('2002-03-30'), dynamic=False)
pred_ci = pred.conf_int()

In [ ]:
pred_ci.head(5)

In [ ]:
plt.close()
axis = co2_levels['1990':].plot(figsize=(10, 6))
pred.predicted_mean.plot(ax=axis, label='One-step ahead Forecast', alpha=0.7)
axis.fill_between(pred_ci.index, pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color='k', alpha=.25)
axis.set_xlabel('Date')
axis.set_ylabel('CO2 Levels')
plt.legend(loc='best')
plt.show()

Copyright 2022. Elaborado por Luis Cajachahua bajo licencia MIT